In [1]:
import os
import json
import pickle

from collections import Counter

import pandas as pd
import numpy as np

# Setup

In [2]:
data_dir = os.path.join(os.curdir, "data")

raw_dataset_url = "https://shai-nlp-course.netlify.app/clean-tweets.tsv"

In [3]:
raw = pd.read_csv(filepath_or_buffer=raw_dataset_url, sep="\t")

raw.head()

,Tweet,Country,Topic,Sentiment,Sentiment_Expression,Sentiment_Target,word_count,char_count,clean_text,clean_stemmed
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",lebanon,personal,negative,implicit,بريق العيون,23,132,اومن بان الانسان ينطفي جماله ابتعاد يحب بريق ا...,اوم بان انس نطف جمل بعد يحب برق عين خفي صبح ذب...
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,jordan,sports,positive,explicit,افضل لاعب في العالم,23,141,الذاكره عندما اعتقد كريستيانو انه افضل لاعب ال...,ذكر عند عقد كريستيانو انه فضل لعب علم ككا يسي ...
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,palestine,personal,neutral,none,none,24,133,نخلو ضغوطات الحياه فنحن نعيش ارض اعدت للبلاء و...,خلو ضغط حيه فنح نعش ارض اعد بلء ولم سلم بيء وك...
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,lebanon,personal,negative,explicit,مصطلحات_لبنانيه,23,135,بتوصل عالبيت بنط بقلك جيت بتقعد لتتحدث معو بقل...,وصل علب بنط بقل جيت قعد حدث معو بقل شو تقم تمش...
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,palestine,personal,negative,explicit,س لم تعد كما ك,16,67,نصمت لتسير حياتنا يرام فالناس تعد كانت نقيه,نصم تسر حيت يرم لنس تعد كانت نقه


In [5]:
tweets = raw["clean_text"]

# Tokenization

First step is to split the dataset into small bits, each bit is called a `token`

once the corpus is tokenized, we can assign each unique `token` an `index`, note that the index value for any token is not important 

In [6]:
sample = tweets.iloc[0]
sample

'اومن بان الانسان ينطفي جماله ابتعاد يحب بريق العيون يختفي فيصبح ذابلا منطفيا يتحول ربيعه خريف'

Tokenizing have many forms, examples: 
- Word Level Tokenization
- Character Level Tokenization
- Sub-word Tokenization
- Byte Pair Encoding

In this chapter we will cover word level and character level tokenization, others will be covered in later chapters

In [7]:
# Implement a generic Tokenizer class
# Different tokenizers will inherit from this class
class Tokenizer:
    def __init__(self, corpus: list[str], min_frequency: int = None):
        self.min_frequency = min_frequency
        self.vocab = self._create_vocab(corpus=corpus)
        
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        ...
    
    def _tokenize_document(self, document: str) -> list[int]:
        ...
    
    def tokenize(self, documents: list[str]) -> list[list[int]]:
        return [self._tokenize_document(document) for document in documents]
    
    def __len__(self):
        return self.vocab

## Word Level

Here we split the sentence into indivual words, omitting the whitespaces between them

In [8]:
sample_tokens = sample.split(" ")
sample_tokens

['اومن',
 'بان',
 'الانسان',
 'ينطفي',
 'جماله',
 'ابتعاد',
 'يحب',
 'بريق',
 'العيون',
 'يختفي',
 'فيصبح',
 'ذابلا',
 'منطفيا',
 'يتحول',
 'ربيعه',
 'خريف']

In [9]:
# Create a list of all unique tokens in the corpus
all_word_tokens = [token for sample in tweets for token in sample.split(" ")]

word_level_tokens = set(all_word_tokens)
print(len(word_level_tokens), len(all_word_tokens))

19644 62252


In [10]:
word2idx = {token: index for index, token in enumerate(word_level_tokens)}
word2idx

{'': 0,
 'رايتنا': 1,
 'لغاوي': 2,
 'وتامر': 3,
 'بالمرتدين': 4,
 'تعكر': 5,
 'يفز': 6,
 'احط': 7,
 'والسيد': 8,
 'نفسيه': 9,
 'نرجع': 10,
 'السر': 11,
 'روجاتك': 12,
 'يبريك': 13,
 'بعدما': 14,
 'اگله': 15,
 'اعطو': 16,
 'قدام': 17,
 '😬': 18,
 'خلال': 19,
 'محزم': 20,
 'هالهدف': 21,
 'مسافه': 22,
 'كليه': 23,
 'لنفسي': 24,
 'الابداع': 25,
 'لغد': 26,
 'العك': 27,
 'ياكلام': 28,
 'تضامن': 29,
 'سمعت': 30,
 'وللجمال': 31,
 'روبهصادف': 32,
 'بيضل': 33,
 'مواعيد': 34,
 'وعدي': 35,
 'للقطاعين': 36,
 'الظباط': 37,
 'وتطالبه': 38,
 'باللي': 39,
 'سليما': 40,
 'هالحيز': 41,
 'القطاع': 42,
 'ترتوتييين': 43,
 'والرقم': 44,
 'والحجز': 45,
 'الدستوري': 46,
 'سموني': 47,
 'انظر': 48,
 'مشروعه': 49,
 'طالعين': 50,
 'الاصه': 51,
 'المهنيه': 52,
 'قالتلو': 53,
 'وازدهاره': 54,
 'ذاتك': 55,
 'وحگ': 56,
 'مضيع': 57,
 'وذالك': 58,
 'اظلمك': 59,
 'العقيق': 60,
 'يباركولي': 61,
 'شوشو': 62,
 'خارج': 63,
 'جاتكو': 64,
 'الازمه': 65,
 'والخنكه': 66,
 'ليسو': 67,
 'ماذيها': 68,
 'قصف': 69,
 'تسيل': 70,
 '🏾':

In [11]:
list(enumerate(Counter(["a", "b", "a", "c"]).items()))

[(0, ('a', 2)), (1, ('b', 1)), (2, ('c', 1))]

In [12]:
class WordLevelTokenizer(Tokenizer):
    def __init__(self, corpus: list[str], min_frequency: int = 0):
        super().__init__(corpus=corpus, min_frequency=min_frequency)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        tokens_counter = Counter([token for sample in corpus for token in sample.split(" ")])
        tokens = [token for token, count in tokens_counter.items() if count >= self.min_frequency]
        vocab = {token: index for index, token in enumerate(tokens, start=2)} 
        vocab["[PAD]"] = 0
        vocab["[OOV]"] = 1
        return vocab
    
    def _tokenize_document(self, document: str) -> list[int]:
        return [self.vocab.get(token, -1) for token in document.split(" ")]

    

In [13]:
word_level_tokenizer = WordLevelTokenizer(corpus=tweets, min_frequency=2)

In [14]:
word_level_tokenizer.vocab

{'اومن': 2,
 'بان': 3,
 'الانسان': 4,
 'ينطفي': 5,
 'جماله': 6,
 'ابتعاد': 7,
 'يحب': 8,
 'بريق': 9,
 'العيون': 10,
 'يختفي': 11,
 'فيصبح': 12,
 'ذابلا': 13,
 'منطفيا': 14,
 'يتحول': 15,
 'ربيعه': 16,
 'خريف': 17,
 'الذاكره': 18,
 'عندما': 19,
 'اعتقد': 20,
 'كريستيانو': 21,
 'انه': 22,
 'افضل': 23,
 'لاعب': 24,
 'العالم': 25,
 'كاكا': 26,
 'ميسي': 27,
 'ثالثا': 28,
 'حدث': 29,
 'العكس': 30,
 '😂': 31,
 'نخلو': 32,
 'ضغوطات': 33,
 'الحياه': 34,
 'فنحن': 35,
 'نعيش': 36,
 'ارض': 37,
 'اعدت': 38,
 'للبلاء': 39,
 'ولم': 40,
 'يسلم': 41,
 'الانبياء': 42,
 'توكل': 43,
 'دايما': 44,
 'وكن': 45,
 'مطمينا': 46,
 'وواثقا': 47,
 'بالله': 48,
 'بتوصل': 49,
 'عالبيت': 50,
 'بنط': 51,
 'بقلك': 52,
 'جيت': 53,
 'بتقعد': 54,
 'لتتحدث': 55,
 'معو': 56,
 'شو': 57,
 'بتقوم': 58,
 'لتمشي': 59,
 'ناسي': 60,
 'شي': 61,
 '🤔': 62,
 'نصمت': 63,
 'لتسير': 64,
 'حياتنا': 65,
 'يرام': 66,
 'فالناس': 67,
 'تعد': 68,
 'كانت': 69,
 'نقيه': 70,
 'صاحب': 71,
 'السمو': 72,
 'الملكي': 73,
 'الامير': 74,
 'الدكتور': 75,


In [15]:
len(word_level_tokenizer.vocab)

10998

## Character Level Tokenization

In [16]:
sample_tokens = list(sample)
sample_tokens

['ا',
 'و',
 'م',
 'ن',
 ' ',
 'ب',
 'ا',
 'ن',
 ' ',
 'ا',
 'ل',
 'ا',
 'ن',
 'س',
 'ا',
 'ن',
 ' ',
 'ي',
 'ن',
 'ط',
 'ف',
 'ي',
 ' ',
 'ج',
 'م',
 'ا',
 'ل',
 'ه',
 ' ',
 'ا',
 'ب',
 'ت',
 'ع',
 'ا',
 'د',
 ' ',
 'ي',
 'ح',
 'ب',
 ' ',
 'ب',
 'ر',
 'ي',
 'ق',
 ' ',
 'ا',
 'ل',
 'ع',
 'ي',
 'و',
 'ن',
 ' ',
 'ي',
 'خ',
 'ت',
 'ف',
 'ي',
 ' ',
 'ف',
 'ي',
 'ص',
 'ب',
 'ح',
 ' ',
 'ذ',
 'ا',
 'ب',
 'ل',
 'ا',
 ' ',
 'م',
 'ن',
 'ط',
 'ف',
 'ي',
 'ا',
 ' ',
 'ي',
 'ت',
 'ح',
 'و',
 'ل',
 ' ',
 'ر',
 'ب',
 'ي',
 'ع',
 'ه',
 ' ',
 'خ',
 'ر',
 'ي',
 'ف']

In [17]:
char_level_tokens = [token for tweet in tweets for token in tweet]

char_level_tokens = set(char_level_tokens)
print(len(char_level_tokens))

325


In [18]:
char2idx = {token: index for index, token in enumerate(char_level_tokens)}
char2idx

{'🌺': 0,
 '💷': 1,
 '🔺': 2,
 '♀': 3,
 '😬': 4,
 'ี': 5,
 'ﺨ': 6,
 '»': 7,
 'ﺑ': 8,
 'ﺬ': 9,
 'ﻚ': 10,
 '😃': 11,
 'ح': 12,
 '🏼': 13,
 '❤': 14,
 'گ': 15,
 'ء': 16,
 '🏾': 17,
 '😨': 18,
 'ﻐ': 19,
 '🕯': 20,
 'ص': 21,
 '‼': 22,
 '♓': 23,
 '🤷': 24,
 'ﻖ': 25,
 '🖕': 26,
 '😙': 27,
 '💤': 28,
 '🇦': 29,
 'ﻱ': 30,
 'ﺫ': 31,
 '💓': 32,
 'ث': 33,
 '⭕': 34,
 '🐐': 35,
 '⠀': 36,
 'ﻓ': 37,
 'ﻗ': 38,
 '·': 39,
 '😅': 40,
 '✨': 41,
 '✊': 42,
 '🤨': 43,
 '🏴': 44,
 '🏃': 45,
 '💳': 46,
 '👻': 47,
 '￼': 48,
 '💙': 49,
 '🇪': 50,
 'ب': 51,
 '😟': 52,
 'ﺣ': 53,
 'ۆ': 54,
 '🌸': 55,
 '⁰': 56,
 '퍼': 57,
 'ا': 58,
 '🇲': 59,
 'ﻤ': 60,
 '🙏': 61,
 '🏿': 62,
 '🌈': 63,
 'ۈ': 64,
 '❣': 65,
 ' ': 66,
 '😞': 67,
 'ض': 68,
 '🇶': 69,
 'ﺙ': 70,
 '✅': 71,
 '🌧': 72,
 '↷': 73,
 'ر': 74,
 '🇯': 75,
 '💎': 76,
 '💻': 77,
 '▪': 78,
 '😳': 79,
 '✍': 80,
 '⚠': 81,
 '🦅': 82,
 '😓': 83,
 '주': 84,
 '🍃': 85,
 '💞': 86,
 '🔞': 87,
 '🌹': 88,
 '🙃': 89,
 '😍': 90,
 '࿐': 91,
 '😘': 92,
 '🌿': 93,
 '😩': 94,
 '🤔': 95,
 '🔴': 96,
 '💛': 97,
 '😌': 98,
 '♥': 99,
 '🤥': 100,

In [19]:
# TODO: Implement character level tokenizer
# 1. __init__()
# 2. _create_vocab
# 3. _tokenize_document 
class CharacterLevelTokenizer(Tokenizer):
    ...

> Notice the difference in the vocabulary size, between word level and character level. Why would you choose one over the other? 

## Result of Tokenization

1. List of documents (corpus)
2. Each document is represented by a sequence of tokens

> Not all documents have the same length

In [20]:
tokenized_tweets = word_level_tokenizer.tokenize(tweets)

In [21]:
max([len(t) for t in tokenized_tweets])

82

# n-grams

To be continued

# Save the Tokenizer

One of the most straight forward ways of saving a Python object is through binary `serialization`

serialization is a method of converting the `object` to `bytes`, these `bytes` can be read later to recreate the object

`pickle` package is the built-in package for object serialization

In [22]:
word_level_tokenizer_path = os.path.join(data_dir, "word-tokenizer.pkl")
word_level_vocab_path = os.path.join(data_dir, "word-level-vocab.json")

with open(word_level_tokenizer_path, "wb+") as f:
    pickle.dump(obj=word_level_tokenizer, file=f)

    
with open(word_level_vocab_path, "wt+") as f:
    json.dump(word_level_tokenizer.vocab, f)

In [23]:
# TODO: Pickle character level tokenizer